In [1]:
import pandas as pd
import numpy as np
from nameparser import HumanName
import re
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

In [2]:
dfroster = pd.read_csv('CornellM_RosterGroup.csv')
dflinkedin = pd.read_csv('CornellMenLinkedIn.csv')

In [3]:
dfroster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 17 columns):
FirstLast      341 non-null object
Criteria       341 non-null object
FirstLast.1    341 non-null object
Year           341 non-null int64
YearsPlayed    341 non-null int64
RosterLink     81 non-null object
Hometown       228 non-null object
HS             225 non-null object
ClassYear      228 non-null object
College        341 non-null object
First          341 non-null object
Gender         341 non-null object
Last           341 non-null object
MidInit        105 non-null object
MidName        6 non-null object
Name           228 non-null object
RosterYear     228 non-null object
dtypes: int64(2), object(15)
memory usage: 45.4+ KB


In [4]:
dfroster['Gender'] = dfroster['Gender'] == 'M'
dfroster['Gender'] = dfroster['Gender'].astype(int)

In [5]:
dflinkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 8 columns):
Name            271 non-null object
LName           271 non-null object
SearchResult    271 non-null object
Headline        271 non-null object
Location        271 non-null object
Company         259 non-null object
School          271 non-null object
Activites       271 non-null object
dtypes: object(8)
memory usage: 17.1+ KB


In [6]:
dflinkedin.head()

,Name,LName,SearchResult,Headline,Location,Company,School,Activites
0,['Rohan Sikka'],Rohan Sikka,https://www.linkedin.com/in/rohan-sikka-b65032...,Incoming Private Equity Analyst at StepStone G...,"London, United Kingdom",Ares Management Corporation,['Cornell University'],[]
1,['Konstantin Kirovski'],Konstantin Kirovski,https://www.linkedin.com/in/konstantin-kirovsk...,Student at Cornell University,Slovenia,NaN,['Cornell University'],[]
2,['Alafia Ayeni'],Alafia Ayeni,https://www.linkedin.com/in/alafia-ayeni-b2326...,Student-Athlete at Cornell University,"San Diego, California",NaN,"['Cornell University', 'Westview High School']","[""Member of Cornell Men's Tennis""]"
3,['Pietro Rimondini'],Pietro Rimondini,https://www.linkedin.com/in/pietro-rimondini-7...,Student at Cornell University,"Ithaca, New York",Diadem Sports,"['Cornell University', 'Cornell University']",[]
4,['Christopher Levine'],Chris LeVine,https://www.linkedin.com/in/chrislevine/,Managing Director Institutional Fixed Income S...,Greater New York City Area,Jefferies,"['Cornell University', 'Cornell University']",[]


In [7]:
# Remove the brackets and single-quotes from the 'Name' string in each cell of the column 
dflinkedin['Name'] = dflinkedin['Name'].str.strip('[]').str.strip("''")


In [8]:
#dflinkedin['Name']

In [9]:
# First remove the brackets from the 'Name' string, then remove single quotes
# Optionally remove double space from Middle Initial/Name calc 
#dflinkedin['Name'] = dflinkedin['Name'].str.strip('[]').astype(str)
#dflinkedin['Name'] = dflinkedin['Name'].str.strip("''").astype(str)
#dflinkedin['Name'] = dflinkedin['Name'].str.replace('  ', ' ')

In [10]:
dflinkedin.head()

,Name,LName,SearchResult,Headline,Location,Company,School,Activites
0,Rohan Sikka,Rohan Sikka,https://www.linkedin.com/in/rohan-sikka-b65032...,Incoming Private Equity Analyst at StepStone G...,"London, United Kingdom",Ares Management Corporation,['Cornell University'],[]
1,Konstantin Kirovski,Konstantin Kirovski,https://www.linkedin.com/in/konstantin-kirovsk...,Student at Cornell University,Slovenia,NaN,['Cornell University'],[]
2,Alafia Ayeni,Alafia Ayeni,https://www.linkedin.com/in/alafia-ayeni-b2326...,Student-Athlete at Cornell University,"San Diego, California",NaN,"['Cornell University', 'Westview High School']","[""Member of Cornell Men's Tennis""]"
3,Pietro Rimondini,Pietro Rimondini,https://www.linkedin.com/in/pietro-rimondini-7...,Student at Cornell University,"Ithaca, New York",Diadem Sports,"['Cornell University', 'Cornell University']",[]
4,Christopher Levine,Chris LeVine,https://www.linkedin.com/in/chrislevine/,Managing Director Institutional Fixed Income S...,Greater New York City Area,Jefferies,"['Cornell University', 'Cornell University']",[]


In [11]:
dflinkedin['fuzz_ratio'] = dflinkedin.apply(lambda x: fuzz.ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_pratio'] = dflinkedin.apply(lambda x: fuzz.partial_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_tokensort'] = dflinkedin.apply(lambda x: fuzz.token_sort_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_tokenset'] = dflinkedin.apply(lambda x: fuzz.token_set_ratio(x['Name'], x['LName']), axis=1)
dflinkedin['fuzz_wratio'] = dflinkedin.apply(lambda x: fuzz.WRatio(x['Name'], x['LName']), axis=1)


In [12]:
# Hardcode College value "Cornell" but later merge roster wtih LinkedIn and use field from Roster
dflinkedin['tennis_yn'] = dflinkedin['Activites'].str.lower().str.contains("tennis")
dflinkedin['School_yn'] = dflinkedin['School'].str.contains("Cornell")
dflinkedin['SameName_yn'] = (dflinkedin['Name'] == dflinkedin['LName'])
dflinkedin['SameName_yn'][dflinkedin['Name'].isna()] = np.nan
dflinkedin.tennis_yn = dflinkedin.tennis_yn.astype(int)
dflinkedin.School_yn = dflinkedin.School_yn.astype(int)
dflinkedin.SameName_yn = dflinkedin.SameName_yn.astype(int)
dflinkedin['SameTennis_yn'] = dflinkedin['tennis_yn'] + dflinkedin['SameName_yn'] 

C:\Users\19145\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
dflinkedin.rename(columns={'Name':'FirstLast'}, inplace=True)

In [14]:
dflinkedin.describe()

,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn
count,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000
mean,79.040590,78.726937,80.608856,82.428044,83.055351,0.302583,0.841328,0.490775,0.793358
std,27.330254,26.876383,26.084374,25.236390,24.178056,0.460226,0.366046,0.500840,0.794490
min,11.000000,9.000000,13.000000,13.000000,16.000000,0.000000,0.000000,0.000000,0.000000
25%,61.500000,64.000000,65.500000,70.000000,72.500000,0.000000,1.000000,0.000000,0.000000
50%,93.000000,100.000000,100.000000,100.000000,100.000000,0.000000,1.000000,0.000000,1.000000
75%,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,2.000000


In [15]:
dflinkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 17 columns):
FirstLast         271 non-null object
LName             271 non-null object
SearchResult      271 non-null object
Headline          271 non-null object
Location          271 non-null object
Company           259 non-null object
School            271 non-null object
Activites         271 non-null object
fuzz_ratio        271 non-null int64
fuzz_pratio       271 non-null int64
fuzz_tokensort    271 non-null int64
fuzz_tokenset     271 non-null int64
fuzz_wratio       271 non-null int64
tennis_yn         271 non-null int32
School_yn         271 non-null int32
SameName_yn       271 non-null int32
SameTennis_yn     271 non-null int32
dtypes: int32(4), int64(5), object(8)
memory usage: 31.9+ KB


In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [17]:
df_suffix = pd.merge(dfroster, dflinkedin,
                     left_on='FirstLast', right_on='FirstLast', how='outer',suffixes=('_R','_L'))

In [18]:
df_suffix['Decade'] = df_suffix['Year']//10*10

In [19]:
df_suffix.drop(columns=['FirstLast.1'], inplace=True)

In [20]:
df_suffix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 340
Data columns (total 33 columns):
FirstLast         341 non-null object
Criteria          341 non-null object
Year              341 non-null int64
YearsPlayed       341 non-null int64
RosterLink        81 non-null object
Hometown          228 non-null object
HS                225 non-null object
ClassYear         228 non-null object
College           341 non-null object
First             341 non-null object
Gender            341 non-null int32
Last              341 non-null object
MidInit           105 non-null object
MidName           6 non-null object
Name              228 non-null object
RosterYear        228 non-null object
LName             271 non-null object
SearchResult      271 non-null object
Headline          271 non-null object
Location          271 non-null object
Company           259 non-null object
School            271 non-null object
Activites         271 non-null object
fuzz_ratio        271 non-n

In [21]:
#df_suffix.to_csv('RosterLinkedIn_CornellM.csv', index=False)

In [22]:
df_suffix.sort_values('Year')

,FirstLast,Criteria,Year,YearsPlayed,RosterLink,Hometown,HS,ClassYear,College,First,Gender,Last,MidInit,MidName,Name,RosterYear,LName,SearchResult,Headline,Location,Company,School,Activites,fuzz_ratio,fuzz_pratio,fuzz_tokensort,fuzz_tokenset,fuzz_wratio,tennis_yn,School_yn,SameName_yn,SameTennis_yn,Decade
113,John Smith,Cornell,1947,1,NaN,"Binghamton, N.Y.",Central H.,['Senior'],Cornell University,John,1,Smith,V.,NaN,John V. Smith,"[""1946-47 Men's Tennis Roster""]",john smith,https://www.linkedin.com/in/john-smith-71730b67/,Student at Cornell University,"Brooklyn, New York",Not Found,['Cornell University'],[],80.0,80.0,100.0,100.0,100.0,0.0,1.0,0.0,0.0,1940
67,Edward Gilbert,Cornell,1947,1,NaN,"Huntington, N.Y.",Horace Mann HS,['Sophomore'],Cornell University,Edward,1,Gilbert,H.,NaN,Edward H. Gilbert,"[""1946-47 Men's Tennis Roster""]",Ed Gilbert,https://www.linkedin.com/in/ed-gilbert-466b412/,Human Resources Consultant,Greater Minneapolis-St. Paul Area,SCORE,['Cornell University'],[],83.0,90.0,83.0,83.0,83.0,0.0,1.0,0.0,0.0,1940
117,Jordan Dingle,Cornell,1947,1,NaN,"Dover, Del.",Dover H.,['Senior'],Cornell University,Jordan,1,Dingle,K.,NaN,Jordan K. Dingle,"[""1946-47 Men's Tennis Roster""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940
187,Robert Rubin,Cornell,1947,1,NaN,"Washington, DC.",Batto Poly.,['Junior'],Cornell University,Robert,1,Rubin,J.,NaN,Robert J. Rubin,"[""1946-47 Men's Tennis Roster""]",Robert Rubin,https://www.linkedin.com/in/robert-rubin-0a9a008/,"Special Counsel at McCarter & English, LLP",Greater New York City Area,"McCarter & English, LLP","['Cornell University', 'Columbia University Sc...",['Sigma Alpha Mu\nChi Epsilon'],100.0,100.0,100.0,100.0,100.0,0.0,1.0,1.0,1.0,1940
56,David Potter,Cornell,1947,1,NaN,"E. Rochester, NY",E. Rochester,['Junior'],Cornell University,David,1,Potter,F.,NaN,David F. Potter,"[""1946-47 Men's Tennis Roster""]",Dave Potter,https://www.linkedin.com/in/dave-potter-33130633/,Independent Farming Professional,"Richland/Kennewick/Pasco, Washington Area",Wheatland Express,"['Cornell University', 'Whitworth University']",[],87.0,82.0,87.0,87.0,87.0,0.0,1.0,0.0,0.0,1940
178,Richard Gnaedinger,Cornell,1947,1,NaN,"Oak Park, Ill.",Oak Park H.,['Freshman'],Cornell University,Richard,1,Gnaedinger,C.,NaN,Richard C. Gnaedinger,"[""1946-47 Men's Tennis Roster""]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940
86,Hollis Young,Cornell,1948,2,NaN,"Brookville, NY.",Hill School,['Junior' 'Senior'],Cornell University,Hollis,1,Young,D.,NaN,Hollis D. Young,"[""1946-47 Men's Tennis Roster"" ""1947-48 Men's ...",Melinda Reyes,https://www.linkedin.com/in/melinda-reyes-7714...,Payment Processing Solutions Expert - with ove...,"Hollis, New York",Premier Merchant Consulting Services,"['Cornell University', 'Jamaica High School']",['Public Relations Student Society of America\...,24.0,25.0,32.0,32.0,32.0,0.0,1.0,0.0,0.0,1940
162,Paul Lansaw,Cornell,1948,2,NaN,"Middletown, Ohio",Middletown H.,['Freshman' 'Sophomore'],Cornell University,Paul,1,Lansaw,NaN,NaN,Paul Lansaw,"[""1946-47 Men's Tennis Roster"" ""1947-48 Men's ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940
189,Robert Corrigan,Cornell,1948,2,NaN,"Baltimore, MD.",Northwestern College,['Freshman' 'Sophomore'],Cornell University,Robert,1,Corrigan,W.,NaN,Robert W. Corrigan,"[""1946-47 Men's Tennis Roster"" ""1947-48 Men's ...",Grant Marani,https://www.linkedin.com/in/grant-marani-4195056/,"Partner, Robert A.M. Stern Architects",Greater New York City Area,Robert A.M. Stern Architects,"['Cornell University', 'University of Melbourne']",[],44.0,52.0,37.0,37.0,44.0,0.0,1.0,0.0,0.0,1940
190,Rod de Llano,Cornell,1948,2,NaN,"Laredo, Tex.",Laredo H.,['Sophomore' 'Junior'],Cornell University,Rod,1,de Llano,R.,NaN,Rod R. de Llano,"[""1946-47 Men's Tennis Roster"" ""1947-48 Men's ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940
